In [ ]:
using DataFrames
using CSV
using Dates
using ProgressMeter
#using ArchGDAL
using StatsBase

In [ ]:
ENV["COLUMNS"]=500;

# Prepare data from SAM

In [ ]:
## Load SAM Test data:
test_cols = [:ctTestDate,:ctTestType_FOI,:ctResultOfTest,:ctClearFlag,:ctAssetPK,:ctCPH,:ctCphh_fmt,:ctNumber,:ctSize,:ctReactors,:ctNumberNotTested,
              :ctBreakId,:ctPartCode,:ctConfirmed,:ctTaken,:ctSHTaken,:ctCategory,:ctInterp,:ctSpecies,:ctSlaughteredIRs]
SAM_Test = CSV.read("/Data/SAM/tblccdTest.txt", DataFrame, dateformat="d/m/yyyy HH:MM:SS", select=test_cols)
;

In [ ]:
## Pull out Gamma tests (for later)
SAM_Test_Gamma = SAM_Test[(SAM_Test.ctTestDate.>=Date("2012")).&&(isequal.(SAM_Test.ctCategory,"GAMMA")),:]  #Using isequal avoids missing
;

In [ ]:
## Select SICCT skin tests from 2012 onwards:
SAM_Test = SAM_Test[(SAM_Test.ctTestDate.>=Date("2012")).&&(isequal.(SAM_Test.ctCategory,"TBSKINTEST")),:]  #Using isequal avoids missing
;

In [ ]:
## Select only Whole Herd tests
#SAM_test_types = CSV.read("/Data/SAM/tlkccdTestType.txt", DataFrame)
#WH_test_types = SAM_test_types[isequal.(SAM_test_types.ttTypeCode,"WH"),:ttCode]

#SAM_Test = 
#SAM_Test = SAM_Test[in(WH_test_types).(SAM_Test.ctTestType_FOI),:]
#;

In [ ]:
## Remove records without a test result
SAM_Test = SAM_Test[(.!ismissing.(SAM_Test.ctResultOfTest)).&&(in(["C","NC"]).(SAM_Test.ctResultOfTest)),:]
;

In [ ]:
## Convert DateTimes to Dates
SAM_Test.ctTestDate = convert.(Date,SAM_Test.ctTestDate)
;

In [ ]:
## Load SAM Herd (Location) data
herd_cols = [:chAssetPK,:chMapX,:chMapY,:chType,:chSpecies,:chHerdSize_CTS]
SAM_Herd = CSV.read("/Data/SAM/tblccdHerd_FOI.txt", DataFrame, select=herd_cols)
;

In [ ]:
SAM_joined = innerjoin(SAM_Test,SAM_Herd,on=:ctAssetPK=>:chAssetPK)
;

In [ ]:
## Load SAM Breakdowns
bd_cols = [:cbAssetPK,:cbBreakDate,:cbConfDate,:cbTB10Date]
SAM_Breakdown = CSV.read("/Data/SAM/tblccdBreakdown.txt", DataFrame, select=bd_cols, delim='|', dateformat="d/m/yyyy HH:MM:SS")
SAM_Breakdown.cbBreakDate = convert.(Date,SAM_Breakdown.cbBreakDate)
SAM_Breakdown.cbConfDate = passmissing(convert).(Date,SAM_Breakdown.cbConfDate)
SAM_Breakdown.cbTB10Date = passmissing(convert).(Date,SAM_Breakdown.cbTB10Date)
;

In [ ]:
## Group breakdowns by asset, for fast searching
SAM_Breakdown_byAsset = groupby(SAM_Breakdown,:cbAssetPK)
;

In [ ]:
## Function: previousBreakdown(assetPK,testDate,breakdownTable)
## Returns the date of the last breakdown before testDate for the asset (herd)
## or "missing" if no previous breakdowns.
## Set pv=2 e.g. to get 2nd last, etc.
## Uses breakdown table grouped by asset for fast searching
function previousBreakdown(assetPK,testDate,pv=1)
    if (assetPK,) in keys( SAM_Breakdown_byAsset)
        bds = SAM_Breakdown_byAsset[(assetPK,)]
    else
        return missing
    end
    pvbds = bds[bds.cbBreakDate.<testDate,:cbBreakDate] |> unique |> sort
    if length(pvbds)>pv-1
        return pvbds[end-pv+1]
    else
        return missing
    end
end
;

In [ ]:
## Add previous breakdown dates to SAM_joined table
@time SAM_joined.previousBreakdown = previousBreakdown.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate)
@time SAM_joined.previousBreakdown2 = previousBreakdown.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate,2)
;    

In [ ]:
## Function: dayValue(x)
## Converts Day type to Int value, handles missing
function dayValue(x::Union{Missing,Day})
    if ismissing(x)
        missing
    else
        x.value
    end
end
;

In [ ]:
## Get number of days since last breakdown
SAM_joined.daysSinceBreakdown = map(dayValue, SAM_joined.ctTestDate - SAM_joined.previousBreakdown)
;

In [ ]:
## Get SAM table grouped by asset, for fast searching
SAM_joined_byAsset = groupby(SAM_joined,:ctAssetPK)
;

In [ ]:
## Function: previousTest(assetPK,testDate)
## Returns the result of the previous test before testDate for the asset (herd)
## or "missing" if no previous tests.
## Set pv=2 e.g. to get 2nd last, etc.
## Uses the SAM table grouped by asset, for fast searching
function previousTest(assetPK,testDate,pv=1)
    tests = SAM_joined_byAsset[(assetPK,)]
    pvtests = tests[tests.ctTestDate.<testDate,:]
    pvtests = sort(unique(pvtests,:ctTestDate),:ctTestDate)
    if nrow(pvtests)>pv-1
        pvtests.ctResultOfTest[end-pv+1]
    else
        missing
    end
end
;

In [ ]:
## Get results of previous tests
@time SAM_joined.previousResultOfTest = previousTest.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate)
@time SAM_joined.previousResultOfTest2 = previousTest.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate,2)
;

In [ ]:
## Function: daysSincePreviousTest(assetPK,testDate)
## Returns the days since previous test before testDate for the asset (herd)
## or "missing" if no previous tests.
## Set pv=2 e.g. to get 2nd last, etc.
## Uses the SAM table grouped by asset, for fast searching
function daysSincePreviousTest(assetPK,testDate,pv=1)
    tests = SAM_joined_byAsset[(assetPK,)]
    pvtests = tests[tests.ctTestDate.<testDate,:]
    pvtests = sort(unique(pvtests,:ctTestDate),:ctTestDate)
    if nrow(pvtests)>pv-1
        dayValue(testDate - pvtests.ctTestDate[end-pv+1])
    else
        missing
    end
end
;

In [ ]:
## Get days since previous tests
@time SAM_joined.daysSincePreviousTest = daysSincePreviousTest.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate)
;

In [ ]:
## Load Defra risk score table from SAM
SAM_RiskScore = CSV.read("/Data/SAM/tblRBT_Scores.txt", DataFrame)
;

In [ ]:
## Group risk scores by cph, for fast searching
SAM_RiskScore_byCPH = groupby(SAM_RiskScore,:CPH)
;

In [ ]:
## Function: defraRiskScore(assetPK,testDate)
## Get the Defra risk score for the holding on the date of the test
function defraRiskScore(cph,testDate)
    if (cph,) in keys(SAM_RiskScore_byCPH)
        scores = SAM_RiskScore_byCPH[(cph,)]
        result = scores.RiskScore[(scores.FromDate.<=testDate).&(scores.ToDate.>=testDate)]
        if length(result) > 0
            return result[end]
        else
            return missing
        end
    else
        return missing
    end
end

In [ ]:
## Get Defra risk score for holding at test date
@time SAM_joined.defraRiskScore = defraRiskScore.(SAM_joined.ctCPH,SAM_joined.ctTestDate)
;

In [ ]:
## Function: breakdownConfirmed(assetPK,testDate,x)
## Returns  whether there was a breakdown within x days of testDate for the asset (herd)
## Uses breakdown table grouped by asset for fast searching
function confirmedBreakdown(assetPK,testDate,x)
    if (assetPK,) in keys(SAM_Breakdown_byAsset)
        bds = SAM_Breakdown_byAsset[(assetPK,)]
    else
        return false
    end
    bds = dropmissing(bds)
    fcbds = bds[(bds.cbConfDate.>=testDate),:cbConfDate]
    cbds = fcbds[fcbds.<=testDate+Day(x)]
    return length(cbds)>0
end
;

In [ ]:
## Get whether the test resulted in a confirmed (lesion or culture)
@time SAM_joined.confirmedBreakdown = confirmedBreakdown.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate,90)
;

# Gamma testing (as proxy for badger culling?)

In [ ]:
## Group gamma tests by asset (for fast searching)
SAM_Test_Gamma_byAsset = groupby(SAM_Test_Gamma,:ctAssetPK)
;

In [ ]:
## Function: gammaCount(assetPK,testDate)
## Gets then number of gamma tests per asset prior to test date
function gammaCount(assetPK,testDate)
    if (assetPK,) in keys(SAM_Test_Gamma_byAsset)
        h = SAM_Test_Gamma_byAsset[(assetPK,)]
        return nrow(h[h.ctTestDate.<=testDate,:])
    else
        return 0
    end
end;

In [ ]:
## Get the number of prior gamma tests in the herd
@time SAM_joined.gammaTestCount = gammaCount.(SAM_joined.ctAssetPK,SAM_joined.ctTestDate)
;

# <mark>Prepare data from CTS

In [ ]:
## Load CTS movements data:
move_cols = [:MovementDate,:OffLocationKey,:OnLocationKey,:Birth,:Death]
CTS_moveT = CSV.read("/Data/CTS/tblMovementTransition.csv", DataFrame, select=move_cols, dateformat="yyyy-mm-dd HH:MM:SS")
;

In [ ]:
## Remove missing dates
dropmissing!(CTS_moveT,:MovementDate)
;

In [ ]:
## Since 2012
CTS_moveT = CTS_moveT[CTS_moveT.MovementDate.>=Date("2012"),:]
;

In [ ]:
## Convert DateTimes to Dates
CTS_moveT.MovementDate = convert.(Date,CTS_moveT.MovementDate)
;

In [ ]:
## Load CTS locations:
loc_cols = [:LocationKey,:CurrentSamCPH]
CTS_loc = CSV.read("/Data/CTS/tblLocation_fixed.csv",DataFrame,select=loc_cols)
;

In [ ]:
## Map CTS Location IDs to CPH (Int)
lockey2cph_map = Dict(zip(CTS_loc.LocationKey, CTS_loc.CurrentSamCPH))
function lockey2cph(x)
    if x in keys(lockey2cph_map)
        lockey2cph_map[x]
    else 
        missing
    end
end
;

In [ ]:
## Add Off/On location CPHs
CTS_moveT.OffCPH = lockey2cph.(CTS_moveT.OffLocationKey)
CTS_moveT.OnCPH = lockey2cph.(CTS_moveT.OnLocationKey)
;

In [ ]:
## Group by CPH (on and off) for fast searching
CTS_moveT_byOnCPH = groupby(CTS_moveT,:OnCPH)
CTS_moveT_byOffCPH = groupby(CTS_moveT,:OffCPH)
;

In [ ]:
## Function: inflow(CPH,testDate,d)
## Gets the number of animals moved onto asset within d days prior to test date
function inflow(cph_i,testDate,d)
    cph = string(cph_i)
    if (cph,) in keys(CTS_moveT_byOnCPH)
        h = CTS_moveT_byOnCPH[(cph,)]
        return nrow(h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)),:])
    else
        return 0
    end
end;

## Function: outflow(CPH,testDate,d)
## Gets the number of animals moved off asset within d days prior to test date
function outflow(cph_i,testDate,d)
    cph = string(cph_i)
    if (cph,) in keys(CTS_moveT_byOffCPH)
        h = CTS_moveT_byOffCPH[(cph,)]
        return nrow(h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)),:])
    else
        return 0
    end
end;

In [ ]:
## Get the number number of animals moved into the herd within d days prior to test
@time SAM_joined.inflow1 = inflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365)
@time SAM_joined.inflow2 = inflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*2)
@time SAM_joined.inflow4 = inflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 356*4)
@time SAM_joined.inflow90 = inflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 90)
;

## Get the number number of animals moved out of the herd within d days prior to test
@time SAM_joined.outflow1 = outflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365)
@time SAM_joined.outflow2 = outflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*2)
@time SAM_joined.outflow4 = outflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*4)
@time SAM_joined.outflow90 = outflow.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 90)
;

In [ ]:
## Function: breakdown_within(assetPK,testDate,d)::Bool
## Does the location have a confirmed breakdown within d days of testDate? (Before or after)
function breakdown_within(loc,testDate,d)
    if ismissing(loc)
        return false
    end
    if (loc,) in keys(SAM_Breakdown_byAsset)
        # get breakdowns at location
        bd = SAM_Breakdown_byAsset[(loc,)]
        # filter missing confirm date
        bd = dropmissing(bd,:cbConfDate)
        # filter confirmed within d days
        bd = bd[(bd.cbConfDate.>=testDate-Day(d)).&(bd.cbConfDate.<testDate+Day(d)),:]
        # are there any?
        return nrow(bd)>0
    else
        return false
    end
end;

In [ ]:
## Function inflow_breakdown(CPH,testDate,d, b)
## Gets the number of animals moved onto the farm, within d days, from farms that had a breakdown within b days.
function inflow_breakdown(cph_i,testDate,d,b)
    cph = string(cph_i)
    if (cph,) in keys(CTS_moveT_byOnCPH)
        # get moves onto location
        h = CTS_moveT_byOnCPH[(cph,)]
        # filter within d days of test date
        h = h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)),:]
        # filter for moves from CPH with breakdown within b days
        h = h[breakdown_within.(h.OffLocationKey,testDate,b),:]
        # return count
        return nrow(h)
    else
        return 0
    end
end;

## Function outflow_breakdown(CPH,testDate,d, b)
## Gets the number of animals moved off the farm, within d days, onto farms that had a breakdown within b days.
function outflow_breakdown(cph_i,testDate,d,b)
    cph = string(cph_i)
    if (cph,) in keys(CTS_moveT_byOffCPH)
        # get moves off location
        h = CTS_moveT_byOffCPH[(cph,)]
        # filter within d days of test date
        h = h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)),:]
        # filter for moves from CPH with breakdown within b days
        h = h[breakdown_within.(h.OnLocationKey,testDate,b),:]
        # return count
        return nrow(h)
    else
        return 0
    end
end;

In [ ]:
## Get the number number of animals moved into the herd within d days prior to test
## from farms that had a breakdown within 2 years
@time SAM_joined.inflowBD1 = inflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365, 365*2)
@time SAM_joined.inflowBD2 = inflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*2, 365*2)
@time SAM_joined.inflowBD4 = inflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*4, 365*2)
@time SAM_joined.inflowBD90 = inflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 90, 365*2)

## Get the number number of animals moved off the herd within d days prior to test
## to farms that had a breakdown within 2 years
@time SAM_joined.outflowBD1 = outflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365, 365*2)
@time SAM_joined.outflowBD2 = outflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*2, 365*2)
@time SAM_joined.outflowBD4 = outflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 365*4, 365*2)
@time SAM_joined.outflowBD90 = outflow_breakdown.(SAM_joined.ctCPH, SAM_joined.ctTestDate, 90, 365*2)
;

## <mark>TODO:</mark>

* ~~Inflow / outflow from breakdown farms~~
* ~~farm type /~~
* ~~test type~~ / risk area
* Age / breed
* Herd on/off restriction
* Defra risk score?
* Seasonality (month of year?)
* Birth/deaths need linking from Animals table (no births in movements, despite column, deaths may be just moves to slaughter?)
* Moves from HRA herds

In [ ]:
## Function: births(CPH,testDate,d)
## Gets the number of animal births onto asset within d days prior to test date
function births(cph,testDate,d)
    if (cph,) in keys(CTS_moveT_byOffCPH)
        h = CTS_moveT_byOffCPH[(cph,)]
        return nrow(h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)).&(h.Birth),:])
    else
        return 0
    end
end;

##

## Function: deaths(CPH,testDate,d)
## Gets the number of animal deaths at asset within d days prior to test date
function deaths(cph,testDate,d)
    if (cph,) in keys(CTS_moveT_byOffCPH)
        h = CTS_moveT_byOffCPH[(cph,)]
        return nrow(h[(h.MovementDate.<=testDate).&(h.MovementDate.>testDate-Day(d)).&(h.Death),:])
    else
        return 0
    end
end;

In [ ]:
## Birth/deaths need linking from Animals table (no births in movements, despite column, deaths may be just moves to slaughter?)

# Add Vet data

In [ ]:
## Load Vet/Tuberculin data from UKFarmcare
Vet_data = CSV.read("/Data/TB_Diagnostics/vetData.csv", DataFrame)
Vet_data_noCat = CSV.read("/Data/TB_Diagnostics/vetData_nonCat.csv", DataFrame) #same without categorisation
;

In [ ]:
## Join the vet data with the SAM data
SAM_joined_noCat = leftjoin(SAM_joined, Vet_data_noCat, on = [:ctTestDate => :Date, :ctCphh_fmt => :CPH], validate=(false, true)) #first without top 250 ctegrisation of vet data
SAM_joined = leftjoin(SAM_joined, Vet_data, on = [:ctTestDate => :Date, :ctCphh_fmt => :CPH], validate=(false, true)) #then with categorisation
;

# Badger data

In [ ]:
# Load badger abundance
badger_data = CSV.read("/Data/TB_Diagnostics/Badgers/badgersAbundancePerHexCell.csv", DataFrame)
# Load cell locations
badger_cells = CSV.read("/Data/TB_Diagnostics/Badgers/locationsPerHexCell.csv", DataFrame)
;

In [ ]:
# match CPHs to badger abundance
badger_cph = leftjoin(badger_cells,badger_data, on=:hexCellID)[:,[:CPH,:meanBadgerAbundance]]
;

In [ ]:
# Join with SAM data
SAM_joined = leftjoin(SAM_joined,badger_cph,on=:ctCPH=>:CPH)
;

In [ ]:
(ismissing.(SAM_joined.meanBadgerAbundance)|>sum) / nrow(SAM_joined)

# Extract features:

In [ ]:
## Function: cat2int(x)
## Transform a vector of categorical values into integers representing each category
function cat2int(v)
    s = Set(v)
    d = Dict(collect(zip(s,1:length(s))))
    map(x->d[x], v)
end

In [ ]:
## Construct Input Vars table
inputVars = DataFrame()

## From SAM
inputVars.dateOfTest = SAM_joined.ctTestDate
inputVars.resultOfTest = SAM_joined.ctResultOfTest.=="NC"
inputVars.monthOfTest = month.(SAM_joined.ctTestDate)
inputVars.severe = SAM_joined.ctInterp.=="SEVERE"
inputVars.animalsTested = SAM_joined.ctNumber
inputVars.locationX = SAM_joined.chMapX
inputVars.locationY = SAM_joined.chMapY
inputVars.previousTestResult = SAM_joined.previousResultOfTest.=="NC"
inputVars.previousTestResult2 = SAM_joined.previousResultOfTest2.=="NC"
inputVars.daysSincePreviousTest = SAM_joined.daysSincePreviousTest
inputVars.daysSinceBreakdown = SAM_joined.daysSinceBreakdown
inputVars.gammaTestCount = SAM_joined.gammaTestCount
inputVars.testType = cat2int(SAM_joined.ctTestType_FOI)
inputVars.herdType = cat2int(SAM_joined.chType)
inputVars.herdSize = SAM_joined.chHerdSize_CTS
inputVars.defraRiskScore = SAM_joined.defraRiskScore

## From CTS
inputVars.inflow1 = SAM_joined.inflow1
inputVars.inflow2 = SAM_joined.inflow2
inputVars.inflow4 = SAM_joined.inflow4
inputVars.inflow90 = SAM_joined.inflow90
inputVars.outflow1 = SAM_joined.outflow1
inputVars.outflow2 = SAM_joined.outflow2
inputVars.outflow4 = SAM_joined.outflow4
inputVars.outflow90 = SAM_joined.outflow90

inputVars.inflowBD1 = SAM_joined.inflowBD1
inputVars.inflowBD2 = SAM_joined.inflowBD2
inputVars.inflowBD4 = SAM_joined.inflowBD4
inputVars.inflowBD90 = SAM_joined.inflowBD90
inputVars.outflowBD1 = SAM_joined.outflowBD1
inputVars.outflowBD2 = SAM_joined.outflowBD2
inputVars.outflowBD4 = SAM_joined.outflowBD4
inputVars.outflowBD90 = SAM_joined.outflowBD90

## From vet data
inputVars.vetPractice = SAM_joined.Practice
inputVars.batchBovine = SAM_joined.BatchBovine
inputVars.batchAvian = SAM_joined.BatchAvian

## Badgers
inputVars.meanBadgerAbundance = SAM_joined.meanBadgerAbundance

## Target var (breakdown within 90 days)
inputVars.confirmedBreakdown = SAM_joined.confirmedBreakdown
;

In [ ]:
## Uncategorised verison
inputVars_noCat = copy(inputVars)
inputVars_noCat.vetPractice = SAM_joined_noCat.Practice
inputVars_noCat.batchBovine = SAM_joined_noCat.BatchBovine
inputVars_noCat.batchAvian = SAM_joined_noCat.BatchAvian
inputVars_noCat.herdType = SAM_joined.chType
inputVars_noCat.testType = SAM_joined.ctTestType_FOI
;

# Write to file

In [ ]:
## Store Input Vars table
CSV.write("/Data/TB_Diagnostics/inputVars.csv",inputVars)
CSV.write("/Data/TB_Diagnostics/inputVars_noCat.csv",inputVars_noCat) #uncategorised version

In [ ]:
## Write file without dates
##CSV.write("/Data/TB_Diagnostics/inputVars_nodate.csv",inputVars[:,2:end])

---
---
# Testing:

In [ ]:
#SAM_Test[SAM_Test.ctTestDate.==Date("2019-01-01"),:]